# Creating Organizer Topics using the API
## Introduction
Algorithmically creating topics using Seeq's REST API can be done using any programming language.  This tutorial will walk through creating these object using Python 2.7 and Seeq's SDK.

The tutorial will show how to create a new Organizer containing a new Topic and modifying the text shown in the topic.  Once this has been accomplished, I recommend that you check out the _Embedding Metrics and Trends using the API_ tutorial to see how to create and include Seeq content.

Note that doing these kinds of operations through the API circumvents all of the front-end's safety checks.  It is **highly recommended** that you develop on a local machine and test all code thoroughly **before using your code on an active server**. You should probably also **back that server up**. As an example of what can go wrong, forgetting the `"`'s in the `workbook_input.data` JSON will render the server unreachable by all...and potentially unfixable!


## Creating an Organizer Topic
Organizer Topics, and Workbench Analyses are handled similarly by Seeq's backend.  Creating and manipulating them is done through the same API endpoint, namely the Workbooks endpoint.  The basic process for creating an Organizer Topic is similar to that for creating a Workbench Analysis, with the addition of creating a document to show in the Topic using the Annotations endpoint.

Creating the Organizer Topic is done in the following steps:
1. Create a new Organizer Topic
1. Update the state of the Organizer Topic as blank. 
1. Create a new Document in the Organizer.
1. Update the "Workstep" for the Document, initializing things like Date Ranges.
1. Create content for the Document.
1. Update the content in the Document.

### Setting up the Environment

In [ ]:
# import the Seeq SDK for communicating with Seeq (I installed this from the egg distributed with Seeq and easy_install)
import seeq_sdk as sdk
# import Jinja2 for creating html templates to post in the document
import jinja2 as ji
# import uuid so that we can create UUID's (aka GUIDs) for our objects
from uuid import uuid4
# import json so that to parse objects
import json
# import datetime for working with times
from datetime import datetime as dt
# import copy so that we can use templates
import copy

### Connecting to Seeq
This is the basic stuff.  Note that these are dummy credentials for a local installation which will not work on your machine or in the cloud.

In [ ]:
# create the API client
api_client = sdk.ApiClient("http://localhost:34216/api", None, None)

# get the clent authoization and login
auth_input = sdk.AuthInputV1(username='seth.gilchrist@seeq.com', password='testpass')
auth_api = sdk.AuthApi(api_client)
auth_output = auth_api.login(body=auth_input)

### Creating a new Organizer Topic
To create an Organizer Topic, we use the same API call that we would to make a new Workbench Analysis, except that we pass the `"isReportBinder": true` flag in the JSON body.

Remember that JSON requires double quotes (`"`) to encapsulate strings.  You can always visit the API help in Seeq (Hamburger Menu -> API Reference) to find out what properties are required.

In [ ]:
topic_input = sdk.WorkbookInputV1()
topic_input.name = 'My First API Created Organizer'
topic_input.data = '{"isReportBinder":true}'

workbook_api = sdk.WorkbooksApi(api_client)
topic_output = workbook_api.create_workbook(body=topic_input)

Once the Organizer Topic has been created, we have to update it and give it a state property using the Items API endpoint.  In this property, we will initialize a "store", which is Seeq terminology for the state of an Analysis or Topic, it is the content of a so-called "Workstep".  Later, we will update the store again through when setting the date ranges in our Organizer Topic. Without setting this property, the workbook will be unreachable through Seeq.

The `version` value is important when passing stores to Seeq. They are used to determine upgrade procedure as the software grows.  They are often the only _required_ property when defining stores through the API. The API will not reject a call without one, but it is important to include them.

In [ ]:
topic_property_input = sdk.PropertyInputV1()
topic_property_input.unit_of_measure = 'string'
topic_property_input.value = '{"version":1,"state":{"stores":{}}}'

items_api = sdk.ItemsApi(api_client)
topic_state_output = items_api.set_property(id=topic_output.id,
                                            property_name='workbookState',
                                            body=topic_property_input)

### Creating a new Document
In this example I will create a new Topic Document, which is done through the Workbook/Worksheets API.  If you have a "template" document, you can duplicate that document by passing its ID in the `branchFrom` property of the `WorksheetInputV1()` object.  To create a new one, you can leave it unspecified or, as I've done here, assign it `None`.

In [ ]:
document_input = sdk.WorksheetInputV1()
document_input.name = 'API Created Document 1'
document_input.branchFrom = None

document_output = workbook_api.create_worksheet(workbook_id=topic_output.id,
                                                body=document_input)

### Create a Workstep
Now that we have a Organizer Topic and document, we need to create a Workstep that will tell Seeq the configuration of the Document. Data about the Workstep configuration are stored in a dataStore.  This isn't the text of the document, but the state of other aspects, such as date ranges or, in the case of a Workbench Analysis, the display range and line colors.

The dataStore is a collection of JSON objects. We can specify any objects that we want to defined and the rest will be populated with default values.  For this example, I am going to define two date ranges that I want stored in the Workstep.

To create the date ranges, I'm going to define a function that takes in lists of start times and end times and populates an empty dateVariable dataStore.  The structure of a dataStore can always be found by looking at the `data` element of a Workstep retrieved through the API reference.

I'll start with the doc string and some constants, including a template of the dateVariable dataStore taken from the Workstep of any Organizer Topic or Workbench Analysis without a defined date range. Next I will move through the input start and end times and populate a new dateVariable dataStore for each pair.  dateVariables are passed in a list in the sqReportStore, hence the final wrapping in `sqReportStore`.  Note the double squiggly brackets (`{{}}`) for escaping when using `format` on JSON objects.  I also return a dictionary with the data range info that is a bit easier to navigate for reference when embedding the Metrics.

In [ ]:
def date_variables_create(start_date, end_date, name=None):
    """
    Creates date variables for a report store.  Input lists of start and end dates to 
    generate a store with multiple date ranges
    Dates should be in ISO 8601 format: yyyy-MM-DDThh:mm:ss.ffffff.
    Note: nanoseconds are not supported, the highest precision in microseconds.
    Note: Timezone is not supported.  Convert to UTC before inputting
    
    :param start_date:  list of start times as ISO 8601 strings
    :param end_date: list of end times as ISO 8601 strings
    :param name: (optional) list of string names for the date ranges. Defaults to "API Created Date Range #"
    :return: the dateVariables with the sqReportStore wrapper
    """

    ms_per_second = 1000
    time_format = '%Y-%m-%dT%H:%M:%S.%f'
    epoch_start = dt(1970, 1, 1)
    default_name = "API Created Date Range {0}"
    date_variable_template = \
        '{\
            "id":"uuid",\
            "name":"name",\
            "range":\
            {\
                "start":0,\
                "end":0\
            },\
            "auto":\
            {\
                "enabled":false,\
                "duration":0,\
                "offset":\
                {\
                    "value":0,\
                    "units":"min"\
                },\
                "offsetDirection":"past",\
                "rate":\
                {\
                    "value":5,\
                    "units":"min"\
                }\
            },\
            "condition":\
            {\
                "strategy":"closestTo",\
                "reference":"end",\
                "offset":1\
            }\
        }'

    date_variable_json = json.loads(date_variable_template)
    
    # If only single start and end dates were passed, wrap them in a list for iteration
    if not isinstance(start_date, list):
        start_date = [start_date]

    if not isinstance(end_date, list):
        end_date = [end_date]
    
    # assign default names if none are given
    if name is None:
        name = [default_name.format(i+1) for i in range(len(start_date))]
    elif not isinstance(name, list):
        name = [name]
    
    assert(len(start_date) == len(name) and len(start_date) == len(end_date))

    date_variables = ''
    date_range_info = {}

    for s, e, n in zip(start_date, end_date, name):
        start_ms_timestamp = \
            (dt.strptime(s, time_format) - epoch_start).total_seconds() * ms_per_second
        end_ms_timestamp =  \
            (dt.strptime(e, time_format) - epoch_start).total_seconds() * ms_per_second
        ID = uuid4()
        current_date_variable = copy.copy(date_variable_json)
        current_date_variable['id'] = str(ID)
        current_date_variable['name'] = n
        current_date_variable['range']['start'] = start_ms_timestamp
        current_date_variable['range']['end'] = end_ms_timestamp        
        # append to the list
        date_variables += (json.dumps(current_date_variable) + ',')
        # add to the info by name
        date_range_info[n] = {'id': ID, 'start_ms': start_ms_timestamp, 'end_ms': end_ms_timestamp,
                             'start_iso': s, 'end_iso': e}

    # remove the trailing comma on the last one
    date_variables = date_variables[:-1]
    
    # return the ReportStore
    return '"sqReportStore":{{"dateVariables":[{0}]}}'.format(date_variables), date_range_info

Now that I can create sqReportStores, I'll create one with two date ranges.  Note that while the Seeq Backend works in nanoseconds, the Frontend works in microseconds, a limitation inherited from Javascript.

In [ ]:
starts = ['2018-12-12T00:00:00.000000', '2019-01-05T13:32:18.934002']
ends = ['2018-12-15T00:00:00.000000', '2019-01-05T15:32:18.934002']

sq_report_store = date_variables_create(starts, ends)

Now that we have our desired starting point stores, I'll create a new Workstep, defining only these stores.  The version number was acquired from the empty dataStore I used as my template.  Again note the double curly braces (`{{}}`) since I'm formating using `format`.

In [ ]:
topic_workstep_input = sdk.WorkstepInputV1()
topic_workstep_input.data = '{{"version":23,"state":{{"stores":{{{0}}}}}}}'.format(sq_report_store[0])

topic_workstep_output = workbook_api.create_workstep(workbook_id=topic_output.id,
                                                     worksheet_id=document_output.id,
                                                     body=topic_workstep_input)

### Creating Content
Next step is to create some content, which we will do through the Annotations API.  The annotation we generate will hold the html that is our document content.  Annotations are used in Seeq for both Topics and Journals, and this one will be created with the type specified as `Report` to indicate it's a Topic Annotation.  The default is `Journal`.

The `interests` list should only have the Worksheet ID of the Topic document.  This tells Seeq to display this Annotation in that document.

In [ ]:
annotation_input = sdk.AnnotationInputV1()

annotations_interested_input = sdk.AnnotationInterestInputV1()
annotations_interested_input.interest_id = document_output.id
annotation_input.interests = [annotations_interested_input]

annotation_input.type = "Report"
annotation_input.name = "Unnammed"

annotations_api = sdk.AnnotationsApi(api_client)
annotation_output = annotations_api.create_annotation(body=annotation_input)

Now that we have the Annotation ID, lets load it with some content.  I'm going to make an html template using Jinja2.  Normally the html template would be specified in an external file, but for this demo, I'm going to define it in-line.

In [ ]:
html_template = r'\
<p>\
    <a href="{{image_source}}">\
        <img src="{{image_source}}" style="width: {{width_percent}}%;\">\
    </a>\
    <br>\
    {{image_caption}}\
</p>'

jinja_environment = ji.Environment(loader=ji.BaseLoader).from_string(html_template)

html_document = jinja_environment.render(
    image_source=r"https://upload.wikimedia.org/wikipedia/commons/thumb/a/a5/Glazed-Donut.jpg/1920px-Glazed-Donut.jpg",
    width_percent=20,
    image_caption="Seeqing for the Breakfast of Champions!!")

When we update the annotation, we must specify the name and re-specify the interested-in list, otherwise the connection between our document and the Annotation will be lost.

In [ ]:
annotation_update_input = sdk.AnnotationInputV1()
annotation_update_input.name = "Unnamed"
annotation_update_input.type = "Report"
annotation_update_input.createdById = "Seth Gilchrist"
annotation_update_input.interests = [annotations_interested_input]
annotation_update_input.document = html_document

annotation_update_output = annotations_api.update_annotation(id=annotation_output.id, body=annotation_update_input)

### Summary of Topic Creation
There you have it!  If you visit Seeq you'll find the document in place, with date ranges as specified, and a picture of everyone's favorite breakfast food.

You can embed many different kinds of content, including Seeq content, into documents using these techniques.  To learn how to include Seeq content in your organizer topic, take a look at the _Embedding Metrics and Trends using the API_ tutorial.

## Contact Us!
If you have questions, comments, or need more help or information, please reach out to your Seeq representative to get a team or 1:1 training on these tools.  We're here to help!